In [1]:
import pandas as pd
import numpy as np

In [2]:
interactions_example = pd.DataFrame([
    {
        "user_id": 1,
        "item_id": 0
    },
    {
        "user_id": 2,
        "item_id": 0
    },
    {
        "user_id": 1,
        "item_id": 4
    },
    {
        "user_id": 1,
        "item_id": 1
    },
    {
        "user_id": 3,
        "item_id": 4
    },
    {
        "user_id": 2,
        "item_id": 1
    },
    {
        "user_id": 4,
        "item_id": 0
    },
    {
        "user_id": 5,
        "item_id": 4
    },
    {
        "user_id": 2,
        "item_id": 4
    },
    {
        "user_id": 4,
        "item_id": 4
    },
    {
        "user_id": 4,
        "item_id": 2
    }
])
interactions_example

,user_id,item_id
0,1,0
1,2,0
2,1,4
3,1,1
4,3,4
5,2,1
6,4,0
7,5,4
8,2,4
9,4,4


In [3]:
items_embeddings = [
    [1, 1, 1, 1, 1],
    [2, 2, 2, 2, 2],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4],
    [5, 5, 5, 5, 5]
]
items_embeddings

[[1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2],
 [3, 3, 3, 3, 3],
 [4, 4, 4, 4, 4],
 [5, 5, 5, 5, 5]]

In [4]:
items_embeddings = np.array(items_embeddings)

In [5]:
users_current_info = {}
contexts = []

for i, row in interactions_example.iterrows():
    user_id = row["user_id"]
    item_id = row["item_id"]

    if user_id not in users_current_info:
        users_current_info[user_id] = {
            'acum_emb': np.array([0, 0, 0, 0, 0]),
            'count': 0
        }
    
    contexts.append(users_current_info[user_id]['acum_emb'] / max(1, users_current_info[user_id]['count']))

    users_current_info[user_id]['acum_emb'] += items_embeddings[item_id]
    users_current_info[user_id]['count'] += 1

contexts

[array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1.]),
 array([3., 3., 3., 3., 3.]),
 array([0., 0., 0., 0., 0.]),
 array([1., 1., 1., 1., 1.]),
 array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([1.5, 1.5, 1.5, 1.5, 1.5]),
 array([1., 1., 1., 1., 1.]),
 array([3., 3., 3., 3., 3.])]

In [6]:
interactions_example['context'] = contexts
interactions_example

,user_id,item_id,context
0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0]"
1,2,0,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,1,4,"[1.0, 1.0, 1.0, 1.0, 1.0]"
3,1,1,"[3.0, 3.0, 3.0, 3.0, 3.0]"
4,3,4,"[0.0, 0.0, 0.0, 0.0, 0.0]"
5,2,1,"[1.0, 1.0, 1.0, 1.0, 1.0]"
6,4,0,"[0.0, 0.0, 0.0, 0.0, 0.0]"
7,5,4,"[0.0, 0.0, 0.0, 0.0, 0.0]"
8,2,4,"[1.5, 1.5, 1.5, 1.5, 1.5]"
9,4,4,"[1.0, 1.0, 1.0, 1.0, 1.0]"


## Testando os modelos MAB usando embeddings de itens

In [7]:
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import plotly.express as px
import time
import os
import implicit
from mab2rec import BanditRecommender, LearningPolicy

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [8]:
FACTORS = 10

In [9]:

def train_embeddings_model(Model, df, num_users, num_items, generate_embeddings=False):
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))

    model = Model(factors=FACTORS, random_state=1, num_threads=1)
    model.fit(sparse_matrix)

    if not generate_embeddings:
        return model, sparse_matrix
    
    # # Não precisamos mais do código abaixo, ele funcina para embeddings de usuário, não de itens
    # user_features_list = []

    # for user_id in df['user_id'].unique():
    #    user_factors = model.user_factors[user_id][:FACTORS]  # O BPR coloca 1 no final dos vetores latentes ?
    #    user_features_list.append([user_id] + list(user_factors))

    # df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(FACTORS)])

    return model, sparse_matrix, model.item_factors

In [10]:

def test_embeddings_model(model, sparse_matrix, df_test):
    all_recs = []

    start_time = time.time()
    hits = 0
    for _, interaction in df_test.iterrows():
        ids_recs, _ = model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1
        all_recs.append(ids_recs.tolist())
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': all_recs
    })
    
    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [11]:

def train_mab(mab_algo, df_train_with_contexts, context_col):
    contexts = np.array(df_train_with_contexts[context_col].tolist())
    mab_algo.fit(
        decisions=df_train_with_contexts['item_id'],
        rewards=df_train_with_contexts['response'],
        contexts=contexts
    )

In [12]:

def test_non_incremental(mab_algo, contexts_col, df_test, interactions_by_user):
    start_time = time.time()
    hits = 0

    # contexts = df_test.merge(user_features, how='left', on='user_id').drop(columns=['user_id', 'item_id', 'response']).values
    contexts = np.array(df_test[contexts_col].tolist())
    filters = df_test.merge(interactions_by_user, how='left', on='user_id').drop(columns=['user_id', 'item_id', 'response', 'als_context', 'bpr_context']).values.squeeze(axis=1) 

    recomendations = mab_algo.recommend(contexts, filters)

    df_test = df_test.reset_index(drop=True)

    hits = 0
    for i, interaction in df_test.iterrows():
        if interaction['item_id'] in recomendations[i]:
            hits += 1
    
    recs_df = pd.DataFrame({
        'interaction_number': [i for i in range(len(df_test))],
        'user_id': df_test['user_id'],
        'item_id': df_test['item_id'],
        'recommendations': recomendations
    })

    return hits, hits/len(df_test), time.time() - start_time, recs_df

In [13]:
def group_interactions_by_user(interactions_df):
    interactions_by_user = interactions_df\
                        .groupby('user_id')[['item_id']]\
                        .apply(lambda df_user: df_user['item_id'].tolist())\
                        .reset_index(name='interactions')
    interactions_by_user = interactions_by_user.reset_index(drop=True)
    return interactions_by_user

In [14]:
def create_contexts_list(interactions_df, items_embeddings):
    users_current_info = {}
    contexts = []

    for _, row in interactions_df.iterrows():
        user_id = row["user_id"]
        item_id = row["item_id"]

        if user_id not in users_current_info:
            users_current_info[user_id] = {
                'acum_emb': np.zeros((FACTORS, )),
                'count': 0
            }
        
        contexts.append(users_current_info[user_id]['acum_emb'] / max(1, users_current_info[user_id]['count']))

        users_current_info[user_id]['acum_emb'] += items_embeddings[item_id][:FACTORS]
        users_current_info[user_id]['count'] += 1

    return contexts

In [15]:

def test(test_size, train_initial_size, train_extra_increment_step_size):
    '''
    - `test_size`: define o tamanho da partição de teste no train/test split inicial. Por exemplo, se for escolhido 0.1 (10%), a partição de teste terá 10% das interações e a partição de treino terá 90% das interações. O tamanho da partição de teste passará ainda por um filtro com o tamanho do treino inicial, definido no próximo parâmetro.
    - `train_initial_size`: define o tamanho inicial que será usado para treino dos modelos. Esse tamanho é uma porcentagem da partição de treino, por exemplo, 0.5 (50%) quer dizer que o treino será feito inicialmente com 50% das interações separadas para treino. Vale ressaltar que essa porcentagem é relacionada apenas à partição de treino, então, se temos uma partição de treino de 0.9 (90%) e o “train_initial_size” é definido como 0.5 (50%), então, teremos 45% (0.9 * 0.5) das interações todas para o treino inicial. Com a base de treino separada com essa porcentagem inicial, a base de teste passara por um filtro, removendo todas as interações com itens ou usuários que nunca foram vistos nesse treino inicial.
    - `train_extra_increment_step_size`: define a porcentagem do "treinamento extra" que será usado. No início a base de dados é separada em treino inicial (train_initial_size), "treinamento extra" e teste. O "treinamento extra", assim como o teste, passa por um filtro para remover interações com itens ou usuários que nunca foram vistos no treino inicial. Após o treino inicial, o "treinamento extra" é usado para treinar os modelos de embeddings e os modelos de bandit. O "treinamento extra" é incrementado a cada iteração, de acordo com o valor desse parâmetro. Por exemplo, se o `train_extra_increment_step_size` é 0.1 (10%), então, a cada iteração, 10% das interações são adicionadas ao treino, até que todo o "treinamento extra" seja usado.
    '''
    results = []
    df_recs = pd.DataFrame(columns=['algorithm', 'interaction_number', 'user_id', 'item_id', 'recommendations'])
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * (1 - test_size))
    df_train_full = df_full[:split_index]
    df_test = df_full[split_index:]

    initial_df_train = df_train_full[:int(len(df_train_full) * train_initial_size)]
    extra_df_train = df_train_full[int(len(df_train_full) * train_initial_size):]
    extra_df_train = extra_df_train[(extra_df_train['user_id'].isin(initial_df_train['user_id'])) & (extra_df_train['item_id'].isin(initial_df_train['item_id']))]
    extra_df_train = extra_df_train.reset_index(drop=True)

    df_test = df_test[(df_test['user_id'].isin(initial_df_train['user_id'])) & (df_test['item_id'].isin(initial_df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    print('Generating ALS embeddings')
    ALS_model, _, ALS_item_embeddings = train_embeddings_model(implicit.als.AlternatingLeastSquares, initial_df_train, num_users, num_items, generate_embeddings=True)

    print('Generating BPR embeddings')
    BPR_model, _, BPR_item_embeddings = train_embeddings_model(implicit.bpr.BayesianPersonalizedRanking, initial_df_train, num_users, num_items, generate_embeddings=True)

    # MUDANCAS AQUI
    df_full_new = pd.concat([initial_df_train, extra_df_train, df_test_for_evaluation])
    als_contexts = create_contexts_list(df_full_new, ALS_item_embeddings)
    df_full_new['als_context'] = als_contexts

    bpr_contexts = create_contexts_list(df_full_new, BPR_item_embeddings)
    df_full_new['bpr_context'] = bpr_contexts

    initial_df_train['als_context'] = als_contexts[:len(initial_df_train)]
    initial_df_train['bpr_context'] = bpr_contexts[:len(initial_df_train)]

    extra_df_train['als_context'] = als_contexts[len(initial_df_train):len(initial_df_train) + len(extra_df_train)]
    extra_df_train['bpr_context'] = bpr_contexts[len(initial_df_train):len(initial_df_train) + len(extra_df_train)]

    df_test_for_evaluation['als_context'] = als_contexts[len(initial_df_train) + len(extra_df_train):]
    df_test_for_evaluation['bpr_context'] = bpr_contexts[len(initial_df_train) + len(extra_df_train):]
    # FIM DAS MUDANÇAS

    def save_algo_result(algo_name, hits, hr, spent_time, df_recs_algo, current_extra_train_size):
        df_recs_algo['algorithm'] = algo_name
        df_recs_algo['train_size'] = current_extra_train_size
        df_recs_new = pd.concat([df_recs, df_recs_algo])
        results.append({'algorithm': algo_name, 'hits': hits, 'hr': hr, 'time': spent_time, 'train_size': current_extra_train_size})
        return df_recs_new

    current_extra_train_size = 0
    while current_extra_train_size <= 1:
        print(f"Current extra train size: {current_extra_train_size}")

        current_df_train = pd.concat([initial_df_train, extra_df_train[:int(len(extra_df_train) * current_extra_train_size)]])
        interactions_by_user = group_interactions_by_user(current_df_train)  # MUDANÇA AQUI

        # -------------- ALS -----------------
        print('Training ALS')
        ALS_model, sparse_matrix = train_embeddings_model(implicit.als.AlternatingLeastSquares, current_df_train, num_users, num_items)

        print('Testing ALS')
        hits, hr, spent_time, df_recs_als = test_embeddings_model(ALS_model, sparse_matrix, df_test_for_evaluation)
        df_recs = save_algo_result('ALS', hits, hr, spent_time, df_recs_als, current_extra_train_size)

        # -------------- BPR -----------------
        print('Training BPR')
        BPR_model, sparse_matrix = train_embeddings_model(implicit.bpr.BayesianPersonalizedRanking, current_df_train, num_users, num_items)

        print('Testing BPR')
        hits, hr, spent_time, df_recs_bpr = test_embeddings_model(BPR_model, sparse_matrix, df_test_for_evaluation)
        df_recs = save_algo_result('BPR', hits, hr, spent_time, df_recs_bpr, current_extra_train_size)

        # ------ LinUCB - ALS embeddings -------
        print('Training LinUCB - ALS embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train_mab(linUCB_model, current_df_train, 'als_context')  # Mudança no treinamento dos MAB

        print('Testing LinUCB - ALS embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_non_incremental(linUCB_model, 'als_context', df_test_for_evaluation, interactions_by_user)
        df_recs = save_algo_result('LinUCB - ALS embeddings', hits, hr, spent_time, df_recs_linUCB, current_extra_train_size)


        # ------ LinUCB - BPR embeddings -------
        print('Training LinUCB - BPR embeddings')
        linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(alpha=0.1), top_k=10)
        train_mab(linUCB_model, current_df_train, 'bpr_context')

        print('Testing LinUCB - BPR embeddings')
        hits, hr, spent_time, df_recs_linUCB = test_non_incremental(linUCB_model, 'bpr_context', df_test_for_evaluation, interactions_by_user)
        df_recs = save_algo_result('LinUCB - BPR embeddings', hits, hr, spent_time, df_recs_linUCB, current_extra_train_size)

        # ------ LinGreedy - ALS embeddings -------
        print('Training LinGreedy - ALS embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train_mab(linGreedy_model, current_df_train, 'als_context')

        print('Testing LinGreedy - ALS embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_non_incremental(linGreedy_model, 'als_context', df_test_for_evaluation, interactions_by_user)
        df_recs = save_algo_result('LinGreedy - ALS embeddings', hits, hr, spent_time, df_recs_linGreedy, current_extra_train_size)


        # ------ LinGreedy - BPR embeddings -------
        print('Training LinGreedy - BPR embeddings')
        linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(epsilon=0.01), top_k=10)
        train_mab(linGreedy_model, current_df_train, 'bpr_context')

        print('Testing LinGreedy - BPR embeddings')
        hits, hr, spent_time, df_recs_linGreedy = test_non_incremental(linGreedy_model, 'bpr_context', df_test_for_evaluation, interactions_by_user)
        df_recs = save_algo_result('LinGreedy - BPR embeddings', hits, hr, spent_time, df_recs_linGreedy, current_extra_train_size)
        
        # Incrementando o tamanho do treino para próxima iteração
        current_extra_train_size = round(current_extra_train_size + train_extra_increment_step_size, 2)
    
    save_path = f'results-v6/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    df_results = pd.DataFrame(results)
    df_results = df_results.astype({'hits': int, 'hr': float, 'time': float})
    df_results['test_size'] = round(test_size, 2)
    df_results['test_interactions'] = len(df_test_for_evaluation)

    df_results.to_csv(f'{save_path}/results.csv', index=False)
    df_recs.to_csv(f'{save_path}/recs.csv', index=False)

In [16]:
test(test_size=0.1, train_initial_size=0.5, train_extra_increment_step_size=0.1)

Generating ALS embeddings


  0%|          | 0/15 [00:00<?, ?it/s]

Generating BPR embeddings


  0%|          | 0/100 [00:00<?, ?it/s]

Current extra train size: 0
Training ALS


...

Testing BPR
Training LinUCB - ALS embeddings
Testing LinUCB - ALS embeddings
Training LinUCB - BPR embeddings
Testing LinUCB - BPR embeddings
Training LinGreedy - ALS embeddings
Testing LinGreedy - ALS embeddings
Training LinGreedy - BPR embeddings
Testing LinGreedy - BPR embeddings
Current extra train size: 1.0
Training ALS


  0%|          | 0/15 [00:00<?, ?it/s]

Testing ALS
Training BPR


  0%|          | 0/100 [00:00<?, ?it/s]

Testing BPR
Training LinUCB - ALS embeddings
Testing LinUCB - ALS embeddings
Training LinUCB - BPR embeddings
Testing LinUCB - BPR embeddings
Training LinGreedy - ALS embeddings
Testing LinGreedy - ALS embeddings
Training LinGreedy - BPR embeddings
Testing LinGreedy - BPR embeddings


In [17]:
df_results = pd.read_csv('results-v6/results.csv')

In [19]:
fig = px.line(df_results, x="train_size", y="hr", color='algorithm', title='HR x Train size')
fig.show()
fig.write_html('results-v6/hr_x_train_size.html')